In [1]:
import sys
sys.executable

'/root/home/envforir/bin/python'

In [80]:
import os
import json
import time
import pandas as pd
import requests

from langchain import hub
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import JSONLoader
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import logging
from openai import OpenAI

In [11]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
UPSTAGE_API_KEY = os.environ.get('UPSTAGE_API_KEY')
LANGCHAIN_API_KEY = os.environ.get('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = 'EXP04_GC' # 프로젝트명 수정
LANGCHAIN_PROJECT = os.environ.get('LANGCHAIN_PROJECT')

print(f'LangSmith Project: {LANGCHAIN_PROJECT}')

LangSmith Project: EXP04_GC


In [18]:
load_dotenv()

True

In [7]:
# 데이터 구성
file_path = '../data/documents.jsonl'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

loader = JSONLoader(
    file_path=file_path,
    jq_schema='.',
    text_content=False,
    json_lines=True,
)
temp = loader.load()

seq_num = 1
documents = []
for tmp in temp:
    data = json.loads(tmp.page_content)
    doc = Document(page_content=data['content'], metadata={
        'docid': data['docid'],
        'src': data['src'],
        'source': '/data/ephemeral/home/upstage-ai-final-ir2/upstage-ai-final-ir2/HM/data/documents.jsonl',
        'seq_num': seq_num,
    })
    documents.append(doc)
    seq_num += 1


In [90]:
# Splitter
splitter = CharacterTextSplitter(
    separator='',
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
)
split_documents = splitter.split_documents(documents)

DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete


In [91]:
# Embedding
embeddings = UpstageEmbeddings(
    api_key=UPSTAGE_API_KEY, 
    model="solar-embedding-1-large"
)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/usr/lib/ssl/certs/ca-certificates.crt'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/usr/lib/ssl/certs/ca-certificates.crt'


In [19]:
# 벡터 저장소 생성
# pip install faiss-cpu
folder_path = f'./faiss_{LANGCHAIN_PROJECT}'
if not os.path.exists(folder_path):
    print('Vector Store 생성 중')
    vectorstore = FAISS.from_documents(
        documents=split_documents,
        embedding=embeddings,
    )
    vectorstore.save_local(folder_path=folder_path)
    print('Vector Store 생성 및 로컬 저장 완료')
else:
    vectorstore = FAISS.load_local(
        folder_path=folder_path, 
        embeddings=embeddings, 
        allow_dangerous_deserialization=True
    )
    print('Vector Store 로컬에서 불러옴')


Vector Store 생성 중
Vector Store 생성 및 로컬 저장 완료


In [20]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
prompt = hub.pull("rlm/rag-prompt")


In [28]:
# LLM과 검색엔진을 활용한 RAG 구현 (기존 코드와 동일)
retriever = vectorstore.as_retriever(k=5)
chat = ChatUpstage(model='solar-1-mini-chat', temperature=0)

In [85]:
# Function Calling 
def is_science_question(question):
    url = "https://api.upstage.ai/v1/solar/chat/completions"
    headers = {
        "Authorization": f"Bearer {UPSTAGE_API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "solar-1-mini-chat",
        "messages": [
            {"role": "system", "content": "You are an AI assistant that determines if a question is about scientific facts. Respond with a JSON object containing 'is_science' (boolean) and 'explanation' (string)."},
            {"role": "user", "content": f"Is this question about scientific facts? Question: {question}"}
        ],
        "temperature": 0
    }

    print(f"Request URL: {url}")
    print(f"Request Headers: {headers}")
    print(f"Request Data: {json.dumps(data, indent=2)}")

    try:
        response = requests.post(url, headers=headers, json=data)
        print(f"Response Status Code: {response.status_code}")
        print(f"Response Content: {response.text}")

        if response.status_code == 200:
            content = response.json()['choices'][0]['message']['content']
            try:
                result = json.loads(content)
                return result
            except json.JSONDecodeError:
                print(f"Failed to parse response: {content}")
                return {"is_science": False, "explanation": "Error in processing"}
        else:
            print(f"API request failed with status code: {response.status_code}")
            return {"is_science": False, "explanation": f"Error in API request: {response.text}"}
    except requests.exceptions.RequestException as e:
        print(f"Request failed with error: {e}")
        return {"is_science": False, "explanation": f"Request error: {str(e)}"}

In [88]:
# 사용 예시
question = "What is the chemical formula for water?"
api_key = "your_api_key_here"
result = is_science_question(question)
print(result)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.upstage.ai:443


Request URL: https://api.upstage.ai/v1/solar/chat/completions
Request Headers: {'Authorization': 'Bearer None', 'Content-Type': 'application/json'}
Request Data: {
  "model": "solar-1-mini-chat",
  "messages": [
    {
      "role": "system",
      "content": "You are an AI assistant that determines if a question is about scientific facts. Respond with a JSON object containing 'is_science' (boolean) and 'explanation' (string)."
    },
    {
      "role": "user",
      "content": "Is this question about scientific facts? Question: What is the chemical formula for water?"
    }
  ],
  "temperature": 0
}


DEBUG:urllib3.connectionpool:https://api.upstage.ai:443 "POST /v1/solar/chat/completions HTTP/1.1" 401 27


Response Status Code: 401
Response Content: {"message": "Unauthorized"}
API request failed with status code: 401
{'is_science': False, 'explanation': 'Error in API request: {"message": "Unauthorized"}'}


In [53]:
# Groundedness Check 함수 추가
def check_groundedness(context, response):
    url = "https://api.upstage.ai/v1/solar/chat/completions"
    headers = {
        "Authorization": f"Bearer {os.environ.get('UPSTAGE_API_KEY')}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "solar-1-mini-groundedness-check",
        "messages": [
            {"role": "user", "content": context},
            {"role": "assistant", "content": response}
        ],
        "temperature": 0
    }
    
    try:
        api_response = requests.post(url, headers=headers, json=data)
        api_response.raise_for_status()  # Raises an HTTPError for bad responses
        result = api_response.json()
        return result['choices'][0]['message']['content']
    except requests.exceptions.RequestException as e:
        print(f"API 요청 오류: {e}")
        return "Error in groundedness check"

In [40]:
def format_docs(docs):
    global references
    references = docs
    return '\n\n'.join(doc.page_content for doc in docs)

In [41]:
def answer_question(messages):
    global references
    response = {"topk": "", "answer": "", "references": "", "groundedness": ""}

    rag_chain = (
        {'context': retriever | format_docs, 'question': RunnablePassthrough()}
        | prompt
        | chat
        | StrOutputParser()
    )

    history = '\n'.join([f"{message['role']}: {message['content']}" for message in messages]) + '\n'
    answer = rag_chain.invoke(history)

    ref_content = [reference.page_content for reference in references]
    topk = [reference.metadata['docid'] for reference in references]

    # Groundedness Check 수행
    context = '\n'.join(ref_content)
    groundedness = check_groundedness(context, answer)

    response["topk"] = topk
    response["answer"] = answer
    response["references"] = ref_content
    response["groundedness"] = groundedness

    return response

In [43]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as eval_lines, open(output_filename, 'w') as output_lines:
        idx = 0
        for eval_line in eval_lines:
            j = json.loads(eval_line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}')
            print(f'Groundedness: {response["groundedness"]}\n')

            output = {
                "eval_id": j["eval_id"], 
                "question": j["msg"][-1]["content"],  # 질문 저장
                "topk": response["topk"], 
                "answer": response["answer"], 
                "references": response["references"],
                "groundedness": response["groundedness"]
            }
            output_lines.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1

In [54]:
# 평가 실행
eval_rag('../data/eval.jsonl', '../output/EXP04_GC.csv')

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
Answer: 나무의 분류에 대해 조사하기 위해서는 생물 분류학에서 사용되는 유사한 특징을 찾아야 합니다. 이에는 성장 속도, 온도 범위, 크기, 잎, 꽃 등이 포함될 수 있습니다. 또한, 생물체의 유전자나 단백질의 구성을 조사하여 분자 수준에서의 분류를 수행할 수도 있습니다. 과학자들은 새로 발견된 생물체를 분류하기 위해 철저한 조사를 진행하며, 생물체의 구조, DNA, 그리고 생활사를 상세히 분석합니다.
Groundedness: grounded

Test 1
Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
Answer: 2017년 현재, 전세계의 공공 교육 지출은 세계 GDP의 약 4%를 차지하고 있습니다. 이는 국가의 교육 체계를 강화하고, 국민들의 교육 기회를 확대하는 데 도움을 줍니다. 많은 국가들이 교육에 많은 예산을 할당하고 있으며, 이는 국가의 교육 수준과 경제 경쟁력을 강화하고자 하는 노력의 일환입니다.
Groundedness: grounded

Test 2
Question: [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'}, {'role': 'assistant', 'content': '네 맞습니다.'}, {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]
Answer: 기억 상실은 대뇌의 기능 장애로 인해 발생할 가능성이 가장 높습니다. 대뇌는 인간의 중추신경계에서 가장 중요한 역할을 담당하며, 인지, 기억, 감정 등 다양한 기능을 조절합니다.
Groundedness: grounded

Test 3
Question: [{'role': 'user', 'content': '통학 버스의 가치에 대해 말해줘.'}]
Answer: 통학 버스는 학생들에게 안전